# EXTRACT

In [13]:
import kagglehub
path = kagglehub.dataset_download("olistbr/brazilian-ecommerce")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\Acer\.cache\kagglehub\datasets\olistbr\brazilian-ecommerce\versions\2


In [14]:
import pandas as pd
import numpy as np

customers = pd.read_csv(f"{path}\olist_customers_dataset.csv")
locations = pd.read_csv(f"{path}\olist_geolocation_dataset.csv")
items = pd.read_csv(f"{path}\olist_order_items_dataset.csv")
payments = pd.read_csv(f"{path}\olist_order_payments_dataset.csv")
reviews = pd.read_csv(f"{path}\olist_order_reviews_dataset.csv")
orders = pd.read_csv(f"{path}\olist_orders_dataset.csv")
products = pd.read_csv(f"{path}\olist_products_dataset.csv")
sellers = pd.read_csv(f"{path}\olist_sellers_dataset.csv")
product_names = pd.read_csv(f"{path}\product_category_name_translation.csv")

# TRANSFORM

### Customers Dataset
This dataset has information about the customer and its location. Use it to identify unique customers in the orders dataset and to find the orders delivery location.
At our system each order is assigned to a unique customer_id. This means that the same customer will get different ids for different orders. The purpose of having a customer_unique_id on the dataset is to allow you to identify customers that made repurchases at the store. Otherwise you would find that each order had a different customer associated with.

customer_id - key to the orders dataset. Each order has a unique customer_id.

customer_unique_id - unique identifier of a customer.

customer_zip_code_prefix - first five digits of customer zip code.

customer_city - customer city name.

customer_state - customer state.

In [3]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [4]:
customers.duplicated().sum()

np.int64(0)

In [5]:
print(customers.shape)
for item in customers.columns:
    print(item, customers[item].nunique(),customers[item].isnull().sum())

(99441, 5)
customer_id 99441 0
customer_unique_id 96096 0
customer_zip_code_prefix 14994 0
customer_city 4119 0
customer_state 27 0


In [6]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


### Geolocation Dataset

This dataset has information Brazilian zip codes and its lat/lng coordinates. Use it to plot maps and find distances between sellers and customers.

geolocation_zip_code_prefix - first 5 digits of zip code.

geolocation_lat - latitude.

geolocation_lng - longitude.

geolocation_city - city name.

geolocation_state - state.

In [7]:
locations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [8]:
locations.duplicated().sum()

np.int64(261831)

In [10]:
locations.drop_duplicates(inplace = True)

In [12]:
print(locations.shape)
for item in locations.columns:
    print(item,locations[item].nunique(),locations[item].isnull().sum())

(738332, 5)
geolocation_zip_code_prefix 19015 0
geolocation_lat 717360 0
geolocation_lng 717613 0
geolocation_city 8011 0
geolocation_state 27 0


In [9]:
locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


### Order Items Dataset
This dataset includes data about the items purchased within each order.

Example:
The order_id = 00143d0f86d6fbd9f9b38ab440ac16f5 has 3 items (same product). Each item has the freight calculated accordingly to its measures and weight. To get the total freight value for each order you just have to sum.
The total order_item value is: 21.33 * 3 = 63.99
The total freight value is: 15.10 * 3 = 45.30
The total order value (product + freight) is: 45.30 + 63.99 = 109.299 = 109.29

order_id - order unique identifier

order_item_id - sequential number identifying number of items included in the same order.

product_id - product unique identifier

seller_id - seller unique identifier

shipping_limit_date - Shows the seller shipping limit date for handling the order over to the logistic partner.

price - item price

freight_value - item freight value item (if an order has more than one item the freight value is splitted between items)


In [10]:
items[items["order_id"] == "00143d0f86d6fbd9f9b38ab440ac16f5"]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.1
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.1
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.1


In [11]:
pd.set_option('display.max_rows',20)
order_counts = items.groupby('order_id').size().reset_index(name='count')
multiple_orders = order_counts[order_counts['count'] > 1]['order_id']
filtered_items = items[items['order_id'].isin(multiple_orders)]
filtered_items.head(20)

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
13,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.90,13.37
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,a17f621c590ea0fab3d5d883e1630ec6,2017-10-20 16:07:52,21.33,15.10
42,001ab0a7578dd66cd4b0a71f5b6e1e41,1,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
44,001ab0a7578dd66cd4b0a71f5b6e1e41,3,0b0172eb0fd18479d29c3bc122c058c2,5656537e588803a555b8eb41f07a944b,2018-01-04 02:33:42,24.89,17.63
48,001d8f0e34a38c37f7dba2a37d4eba8b,1,e67307ff0f15ade43fcb6e670be7a74c,f4aba7c0bca51484c30ab7bdc34bcdd1,2017-05-18 17:35:11,18.99,7.78
49,001d8f0e34a38c37f7dba2a37d4eba8b,2,e67307ff0f15ade43fcb6e670be7a74c,f4aba7c0bca51484c30ab7bdc34bcdd1,2017-05-18 17:35:11,18.99,7.78


In [12]:
pd.set_option('display.max_rows',10)
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [13]:
items.duplicated().sum()

np.int64(0)

In [14]:
print(items.shape)
for item in items.columns:
    print(item, items[item].nunique(),items[item].isnull().sum())

(112650, 7)
order_id 98666 0
order_item_id 21 0
product_id 32951 0
seller_id 3095 0
shipping_limit_date 93318 0
price 5968 0
freight_value 6999 0


In [15]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   order_id             112650 non-null  object 
 1   order_item_id        112650 non-null  int64  
 2   product_id           112650 non-null  object 
 3   seller_id            112650 non-null  object 
 4   shipping_limit_date  112650 non-null  object 
 5   price                112650 non-null  float64
 6   freight_value        112650 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 6.0+ MB


In [16]:
items.describe()

,order_item_id,price,freight_value
count,112650.000000,112650.000000,112650.000000
mean,1.197834,120.653739,19.990320
std,0.705124,183.633928,15.806405
min,1.000000,0.850000,0.000000
25%,1.000000,39.900000,13.080000
50%,1.000000,74.990000,16.260000
75%,1.000000,134.900000,21.150000
max,21.000000,6735.000000,409.680000


In [17]:
# convert date columns into datetime format
items['shipping_limit_date'] = pd.to_datetime(items['shipping_limit_date'])

### Payments Dataset
This dataset includes data about the orders payment options.

order_id - unique identifier of an order.

payment_sequential - a customer may pay an order with more than one payment method. If he does so, a sequence will be 

created to accommodate all payments.

payment_type - method of payment chosen by the customer.

payment_installments - number of installments chosen by the customer.

payment_value - transaction value.

In [18]:
payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [19]:
payments.duplicated().sum()

np.int64(0)

In [20]:
payments["payment_type"].value_counts()

payment_type
credit_card    76795
boleto         19784
voucher         5775
debit_card      1529
not_defined        3
Name: count, dtype: int64

### Order Reviews Dataset
This dataset includes data about the reviews made by the customers.

After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments.

review_id - unique review identifier.

order_id - unique order identifier.

review_score - note ranging from 1 to 5 given by the customer on a satisfaction survey.

review_comment_title - comment title from the review left by the customer, in Portuguese.

review_comment_message - comment message from the review left by the customer, in Portuguese.

review_creation_date - shows the date in which the satisfaction survey was sent to the customer.

review_answer_timestamp - shows satisfaction survey answer timestamp.

In [21]:
reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [22]:
reviews.duplicated().sum()

np.int64(0)

In [23]:
print(reviews.shape)
for item in reviews.columns:
    print(item, reviews[item].nunique(),reviews[item].isnull().sum())

(99224, 7)
review_id 98410 0
order_id 98673 0
review_score 5 0
review_comment_title 4527 87656
review_comment_message 36159 58247
review_creation_date 636 0
review_answer_timestamp 98248 0


In [24]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  int64 
 3   review_comment_title     11568 non-null  object
 4   review_comment_message   40977 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.3+ MB


In [25]:
reviews.drop(columns = ['review_comment_title', 'review_comment_message'], inplace = True)

In [26]:
reviews.head()

,review_id,order_id,review_score,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,2018-03-01 00:00:00,2018-03-02 10:26:53


In [27]:
reviews['review_score'].describe()

count    99224.000000
mean         4.086421
std          1.347579
min          1.000000
25%          4.000000
50%          5.000000
75%          5.000000
max          5.000000
Name: review_score, dtype: float64

In [28]:
reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])

### Order Dataset
This is the core dataset. From each order you might find all other information.

order_id - unique identifier of the order.

customer_id - key to the customer dataset. Each order has a unique customer_id.

order_status - reference to the order status (delivered, shipped, etc.).

order_purchase_timestamp - shows the purchase timestamp.

order_approved_at - shows the payment approval timestamp.

order_delivered_carrier_date - shows the order posting timestamp; when it was handled to the logistic partner.

order_delivered_customer_date - shows the actual order delivery date to the customer.

order_estimated_delivery_date - shows the estimated delivery date that was informed to the customer at the purchase moment.


In [29]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [30]:
orders.duplicated().sum()

np.int64(0)

In [31]:
print(orders.shape)
for item in orders.columns:
    print(item, orders[item].nunique(),orders[item].isnull().sum())

(99441, 8)
order_id 99441 0
customer_id 99441 0
order_status 8 0
order_purchase_timestamp 98875 0
order_approved_at 90733 160
order_delivered_carrier_date 81018 1783
order_delivered_customer_date 95664 2965
order_estimated_delivery_date 459 0


In [32]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [33]:
# convert date columns into datetime format
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders['order_approved_at'] = pd.to_datetime(orders['order_approved_at'])
orders['order_delivered_carrier_date'] = pd.to_datetime(orders['order_delivered_carrier_date'])
orders['order_delivered_customer_date'] = pd.to_datetime(orders['order_delivered_customer_date'])
orders['order_estimated_delivery_date'] = pd.to_datetime(orders['order_estimated_delivery_date'])

In [34]:
orders["order_status"].value_counts()

order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64

### Products Dataset
This dataset includes data about the products sold by Olist.

product_id - unique product identifier.

product_category_name - root category of the product, in Portuguese.

product_name_length - number of characters extracted from the product name.

product_description_length - number of characters extracted from the product description.

product_photos_qty - number of product published photos.

product_weight_g - product weight measured in grams.

product_length_cm - product length measured in centimeters.

product_height_cm - product height measured in centimeters.

product_width_cm - product width measured in centimeters.

In [35]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [36]:
products.duplicated().sum()

np.int64(0)

In [37]:
products = products.merge(product_names, 
                          on='product_category_name', 
                          how='left')

# Replacing the Portuguese category name with the English category name
products['product_category_name'] = products['product_category_name_english']

# Dropping the column with the English category names
products.drop(columns=['product_category_name_english'], inplace=True)

In [38]:
products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [39]:
print(products.shape)
for item in products.columns:
    print(item, products[item].nunique()," - ", products[item].isnull().sum())

(32951, 9)
product_id 32951  -  0
product_category_name 71  -  623
product_name_lenght 66  -  610
product_description_lenght 2960  -  610
product_photos_qty 19  -  610
product_weight_g 2204  -  2
product_length_cm 99  -  2
product_height_cm 102  -  2
product_width_cm 95  -  2


In [40]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32328 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [41]:
# Ensure correct data types in products DataFrame
products.drop(columns = ['product_name_lenght','product_description_lenght'],inplace = True)
products['product_weight_g'] = pd.to_numeric(products['product_weight_g'], errors='coerce')
products['product_length_cm'] = pd.to_numeric(products['product_length_cm'], errors='coerce')
products['product_height_cm'] = pd.to_numeric(products['product_height_cm'], errors='coerce')
products['product_width_cm'] = pd.to_numeric(products['product_width_cm'], errors='coerce')
products['product_photos_qty'] = pd.to_numeric(products['product_photos_qty'], errors='coerce')

In [42]:
products.head()

,product_id,product_category_name,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumery,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,art,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,sports_leisure,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,baby,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,housewares,4.0,625.0,20.0,17.0,13.0


In [43]:
product_names.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [15]:
print(product_names.shape)
for item in product_names.columns:
    print(item, product_names[item].nunique(),product_names[item].isnull().sum())

(71, 2)
product_category_name 71 0
product_category_name_english 71 0


### Sellers Dataset
This dataset includes data about the sellers that fulfilled orders made at Olist. Use it to find the seller location and to identify which seller fulfilled each product.

seller_id - seller unique identifier.

seller_zip_code_prefix - first 5 digits of seller zip code.

seller_city - seller city name.

seller_state - seller state.

In [44]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [45]:
sellers.duplicated().sum()

np.int64(0)

In [46]:
print(sellers.shape)
for item in sellers.columns:
    print(item, sellers[item].nunique()," - ", sellers[item].isnull().sum())

(3095, 4)
seller_id 3095  -  0
seller_zip_code_prefix 2246  -  0
seller_city 611  -  0
seller_state 23  -  0


In [47]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [48]:
ls_df = [customers,locations,items,payments,reviews,orders,products,sellers]

for i in range(len(ls_df)):
    # Приведение всех нулевых значений к NaN
    ls_df[i] = ls_df[i].mask(pd.isna(ls_df[i]), np.nan)
    # Преобразование всех столбцов с датами в datetime, заменяя NaT на NaN
    for column in ls_df[i].select_dtypes(include=['datetime64[ns]']).columns:
        ls_df[i][column] = pd.to_datetime(ls_df[i][column], errors='coerce')
    print(list(ls_df[i].columns),len(ls_df[i].columns))

['customer_id', 'customer_unique_id', 'customer_zip_code_prefix', 'customer_city', 'customer_state'] 5
['geolocation_zip_code_prefix', 'geolocation_lat', 'geolocation_lng', 'geolocation_city', 'geolocation_state'] 5
['order_id', 'order_item_id', 'product_id', 'seller_id', 'shipping_limit_date', 'price', 'freight_value'] 7
['order_id', 'payment_sequential', 'payment_type', 'payment_installments', 'payment_value'] 5
['review_id', 'order_id', 'review_score', 'review_creation_date', 'review_answer_timestamp'] 5
['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp', 'order_approved_at', 'order_delivered_carrier_date', 'order_delivered_customer_date', 'order_estimated_delivery_date'] 8
['product_id', 'product_category_name', 'product_photos_qty', 'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm'] 7
['seller_id', 'seller_zip_code_prefix', 'seller_city', 'seller_state'] 4


In [49]:
df_names = ['customers', 'locations', 'items', 'payments', 'reviews', 'orders', 'products', 'sellers']
ls_df = [customers, locations, items, payments, reviews, orders, products, sellers]

for name, df in zip(df_names, ls_df):
    print(f"DataFrame: {name.upper()}")
    # print(df.shape)
    print(df.info())
    for column in df.columns:
        unique_count = df[column].nunique()
        null_count = df[column].isnull().sum()
        print(f"{column}: {unique_count} unique values - {null_count} missing values")
    print("\n")


DataFrame: CUSTOMERS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB
None
customer_id: 99441 unique values - 0 missing values
customer_unique_id: 96096 unique values - 0 missing values
customer_zip_code_prefix: 14994 unique values - 0 missing values
customer_city: 4119 unique values - 0 missing values
customer_state: 27 unique values - 0 missing values


DataFrame: LOCATIONS
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column           

# LOAD INTO PostgreSQL 

In [50]:
from sqlalchemy import create_engine, text
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

# Database configuration
DB_USERNAME = 'postgres'
DB_PASSWORD = '1234'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'brazilian_ecommerce_db'
DEFAULT_DB = 'postgres'  

# Create an engine to connect to PostgreSQL
engine = create_engine(f'postgresql+psycopg2://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DEFAULT_DB}')

# Create the new database outside of a transaction
with engine.connect() as connection:
    connection.execution_options(isolation_level="AUTOCOMMIT")  # Set autocommit for creating database
    print(f"Creating database '{DB_NAME}'.")
    connection.execute(text(f"CREATE DATABASE {DB_NAME};"))

# Dispose of the engine
engine.dispose()

Creating database 'brazilian_ecommerce_db'.


In [51]:
from sqlalchemy import Column, Integer, String, Float, DateTime
from sqlalchemy.ext.declarative import declarative_base

# Now connect to the newly created database for ORM operations
engine = create_engine(f'postgresql+psycopg2://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}', echo = False)

# Create a base class for ORM models
Base = declarative_base()

class Customer(Base):
    __tablename__ = 'customers'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    customer_id = Column(String, nullable=False)
    customer_unique_id = Column(String, nullable=False)
    customer_zip_code_prefix = Column(Integer, nullable=False)
    customer_city = Column(String, nullable=False)
    customer_state = Column(String, nullable=False)


class Location(Base):
    __tablename__ = 'geolocations'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    geolocation_zip_code_prefix = Column(Integer, nullable=False)
    geolocation_lat = Column(Float, nullable=False)
    geolocation_lng = Column(Float, nullable=False)
    geolocation_city = Column(String, nullable=False)
    geolocation_state = Column(String, nullable=False)


class Item(Base):
    __tablename__ = 'order_items'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    order_id = Column(String, nullable=False)
    order_item_id = Column(Integer, nullable=False)
    product_id = Column(String, nullable=False)
    seller_id = Column(String, nullable=False)
    shipping_limit_date = Column(DateTime, nullable=False)
    price = Column(Float, nullable=False)
    freight_value = Column(Float, nullable=False)


class Payment(Base):
    __tablename__ = 'payments'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    order_id = Column(String, nullable=False)
    payment_sequential = Column(Integer, nullable=False)
    payment_type = Column(String, nullable=False)
    payment_installments = Column(Integer, nullable=False)
    payment_value = Column(Float, nullable=False)


class Review(Base):
    __tablename__ = 'reviews'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    review_id = Column(String, nullable=False)  # Not a primary key
    order_id = Column(String, nullable=False)
    review_score = Column(Integer, nullable=False)
    review_creation_date = Column(DateTime, nullable=False)
    review_answer_timestamp = Column(DateTime, nullable=True)


class Order(Base):
    __tablename__ = 'orders'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    order_id = Column(String, nullable=False)
    customer_id = Column(String, nullable=False)
    order_status = Column(String, nullable=False)
    order_purchase_timestamp = Column(DateTime, nullable=False)
    order_approved_at = Column(DateTime, nullable=True)
    order_delivered_carrier_date = Column(DateTime, nullable=True)
    order_delivered_customer_date = Column(DateTime, nullable=True)
    order_estimated_delivery_date = Column(DateTime, nullable=True)


class Product(Base):
    __tablename__ = 'products'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    product_id = Column(String, nullable=False)
    product_category_name = Column(String, nullable=True)
    product_photos_qty = Column(Float, nullable=True)
    product_weight_g = Column(Float, nullable=True)
    product_length_cm = Column(Float, nullable=True)
    product_height_cm = Column(Float, nullable=True)
    product_width_cm = Column(Float, nullable=True)


class Seller(Base):
    __tablename__ = 'sellers'
    id = Column(Integer, primary_key=True, autoincrement=True)  # Primary key
    seller_id = Column(String, nullable=False)
    seller_zip_code_prefix = Column(Integer, nullable=False)
    seller_city = Column(String, nullable=False)
    seller_state = Column(String, nullable=False)

# Create tables in the database
Base.metadata.create_all(engine)

C:\Users\Acer\AppData\Local\Temp\ipykernel_10644\510119564.py:8: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [52]:
import pandas as pd
from sqlalchemy.orm import sessionmaker

# Create a session
Session = sessionmaker(bind=engine)

# Use a context manager for session handling
with Session() as session:
    try:
        # Cast columns to DateTime where necessary and insert into the database

        customers['customer_zip_code_prefix'] = customers['customer_zip_code_prefix'].astype(int)
        session.bulk_save_objects([
            Customer(
                customer_id=row['customer_id'],
                customer_unique_id=row['customer_unique_id'],
                customer_zip_code_prefix=row['customer_zip_code_prefix'],
                customer_city=row['customer_city'],
                customer_state=row['customer_state']
            )
            for _, row in customers.iterrows()
        ])

        session.bulk_save_objects([
            Location(
                geolocation_zip_code_prefix=row['geolocation_zip_code_prefix'],
                geolocation_lat=row['geolocation_lat'],
                geolocation_lng=row['geolocation_lng'],
                geolocation_city=row['geolocation_city'],
                geolocation_state=row['geolocation_state']
            )
            for _, row in locations.iterrows()
        ])

        session.bulk_save_objects([
            Item(
                order_id=row['order_id'],
                order_item_id=row['order_item_id'],
                product_id=row['product_id'],
                seller_id=row['seller_id'],
                shipping_limit_date=pd.to_datetime(row['shipping_limit_date']),
                price=row['price'],
                freight_value=row['freight_value']
            )
            for _, row in items.iterrows()
        ])

        session.bulk_save_objects([
            Payment(
                order_id=row['order_id'],
                payment_sequential=row['payment_sequential'],
                payment_type=row['payment_type'],
                payment_installments=row['payment_installments'],
                payment_value=row['payment_value']
            )
            for _, row in payments.iterrows()
        ])

        reviews['review_creation_date'] = pd.to_datetime(reviews['review_creation_date'])
        reviews['review_answer_timestamp'] = pd.to_datetime(reviews['review_answer_timestamp'])
        session.bulk_save_objects([
            Review(
                review_id=row['review_id'],
                order_id=row['order_id'],
                review_score=row['review_score'],
                review_creation_date=row['review_creation_date'],
                review_answer_timestamp=row['review_answer_timestamp']
            )
            for _, row in reviews.iterrows()
        ])

        session.bulk_save_objects([
            Order(
                order_id=row['order_id'],
                customer_id=row['customer_id'],
                order_status=row['order_status'],
                order_purchase_timestamp=row['order_purchase_timestamp'],
                order_approved_at=row['order_approved_at'] if pd.notna(row['order_approved_at']) else None,
                order_delivered_carrier_date=row['order_delivered_carrier_date'] if pd.notna(row['order_delivered_carrier_date']) else None,
                order_delivered_customer_date=row['order_delivered_customer_date'] if pd.notna(row['order_delivered_customer_date']) else None,
                order_estimated_delivery_date=row['order_estimated_delivery_date'],
            )
            for _, row in orders.iterrows()
        ])

        session.bulk_save_objects([
            Product(
                product_id=row['product_id'],
                product_category_name=row.get('product_category_name', None),
                product_photos_qty=row.get('product_photos_qty', None),
                product_weight_g=row.get('product_weight_g', None),
                product_length_cm=row.get('product_length_cm', None),
                product_height_cm=row.get('product_height_cm', None),
                product_width_cm=row.get('product_width_cm', None)
            )
            for _, row in products.iterrows()
        ])

        session.bulk_save_objects([
            Seller(
                seller_id=row['seller_id'],
                seller_zip_code_prefix=row['seller_zip_code_prefix'],
                seller_city=row['seller_city'],
                seller_state=row['seller_state']
            )
            for _, row in sellers.iterrows()
        ])

        # Commit all the changes to the database
        session.commit()
        
    
    except Exception as e:
        session.rollback()
        print(f"An error occurred: {e}")

engine.dispose()